# IBM Instance Setup

In [17]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

# Experiments Poster

In [18]:
NODES = 10
SP = "Sparse"
MD = "Medium"
DS = "Dense"


ER = "ER"
BP = "Bipartite"
CM = "Complete"

In [19]:
ER10NSParams = np.load("results/params/params_10N_ER_Sparse_LR0.032372631678972856_SIGMA0.7649320273591023_INIT1.5357950633585402_NUMLAYERS1.npy")
ER10NMParams = np.load("results/params/params_10N_ER_Medium_LR0.007216520881270238_SIGMA0.6960408118792989_INIT0.893668244257591_NUMLAYERS1.npy")
ER10NDParams = np.load("results/params/params_10N_ER_Dense_LR0.031248086322939895_SIGMA0.7735621474524553_INIT0.42238728842048007_NUMLAYERS2.npy")
BP10NSParams = np.load("results/params/params_10N_Bipartite_Sparse_LR0.006115521401940773_SIGMA1.0372657813112909_INIT0.6303388864180629_NUMLAYERS2.npy")
BP10NMParams = np.load("results/params/params_10N_Bipartite_Medium_LR0.0037711868141441346_SIGMA1.5585418217447389_INIT0.416282403403894_NUMLAYERS2.npy")
BP10NDParams = np.load("results/params/params_10N_Bipartite_Dense_LR0.0009131935892470876_SIGMA0.10121114465217869_INIT0.8314880277242851_NUMLAYERS3.npy")

# Hardware test

In [20]:
load_dotenv('.env')

ibm_token = os.getenv('IBM_TOKEN')
instance = os.getenv("INSTANCE")
setup = True 

if setup:
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=ibm_token, overwrite=True)

service = QiskitRuntimeService(channel="ibm_cloud", token = ibm_token, instance=instance)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=156)
print(backend)

<IBMBackend('ibm_aachen')>


In [21]:
dev = qml.device('qiskit.remote', 
                    wires=backend.num_qubits, 
                    backend=backend, 
                    shots=512,
                    optimization_level=3)

## Experiments

In [22]:
QUBITS = NODES * (NODES - 1) // 2
grid_conn = aachen_connectivity()

L1G = efficient_connectivity_gates(grid_conn, QUBITS, 1)
L2G = efficient_connectivity_gates(grid_conn, QUBITS, 2)
L3G = efficient_connectivity_gates(grid_conn, QUBITS, 3)
L4G = efficient_connectivity_gates(grid_conn, QUBITS, 4)
L5G = efficient_connectivity_gates(grid_conn, QUBITS, 5)

L1IQP = iqp.IqpSimulator(QUBITS, L1G, device="lightning.qubit")
L2IQP = iqp.IqpSimulator(QUBITS, L2G, device="lightning.qubit")
L3IQP = iqp.IqpSimulator(QUBITS, L3G, device="lightning.qubit")
L4IQP = iqp.IqpSimulator(QUBITS, L4G, device="lightning.qubit")
L5IQP = iqp.IqpSimulator(QUBITS, L5G, device="lightning.qubit")

@qml.qnode(dev)
def L1Sample(trained_params):
    L1IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L2Sample(trained_params):
    L2IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L3Sample(trained_params):
    L3IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L4Sample(trained_params):
    L4IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L5Sample(trained_params):
    L5IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

In [23]:
with Session(backend=backend, max_time="12h") as session:
    ER10NSRes  = L1Sample(ER10NSParams)
    ER10NMRes  = L1Sample(ER10NMParams)
    ER10NDRes  = L2Sample(ER10NDParams)

    BP10NDRes = L3Sample(BP10NDParams)
    BP10NMRes = L2Sample(BP10NMParams)
    BP10NSRes = L2Sample(BP10NSParams)

In [24]:
np.save("results/samples/2025-06-06/ER10NS.npy", ER10NSRes)
np.save("results/samples/2025-06-06/ER10NM.npy", ER10NMRes)
np.save("results/samples/2025-06-06/ER10ND.npy", ER10NDRes)
np.save("results/samples/2025-06-06/BP10NS.npy", BP10NSRes)
np.save("results/samples/2025-06-06/BP10NM.npy", BP10NMRes)
np.save("results/samples/2025-06-06/BP10ND.npy", BP10NDRes)
print("Samples saved successfully.")

Samples saved successfully.
